In [1]:
from os import chdir
chdir('/home/jovyan/work')

In [2]:
from lib.postgres import connect_to_postgres
import numpy as np

### Attribute Type
    age: continuous
    workclass: class
    fnlwgt: continuous
    education: class
    education_num: continuous
    marital_status: class
    occupation: class
    relationship: class
    race: class
    gender: class
    capital_gain: continuous
    capital_loss: continuous
    hours_per_week: continuous
    native_country: class
    income_label: class

In [3]:
con, cur = connect_to_postgres()
cur.execute("""
BEGIN;
ALTER TABLE adult ADD COLUMN _id SERIAL PRIMARY KEY;
ALTER TABLE adult ADD COLUMN target BOOLEAN;
COMMIT;
""")
con.close()

In [4]:
con, cur = connect_to_postgres()
cur.execute("""SELECT DISTINCT(income_label) FROM adult;""") 
print(cur.fetchall())
con.close()

[(' >50K',), (' <=50K',)]


In [5]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label FROM adult WHERE target IS NULL;""") 
this_id, income_label = cur.fetchone()

greater_than_50k = (income_label == ' >50K')

cur.execute("""
BEGIN;
UPDATE adult 
SET target = {} 
WHERE _id = {};
COMMIT;
""".format(greater_than_50k, this_id)) 

con.close()

In [6]:
con, cur = connect_to_postgres()
cur.execute("""
SELECT _id, income_label, target 
FROM adult WHERE _id = {};
""".format(this_id)) 
print(this_id, cur.fetchone())
con.close()

10 (10, ' >50K', True)


In [7]:
def encode_target(_id):
    """Encode the target for a single row as a boolean value. Takes a row _id."""
    con, cur = connect_to_postgres()
    cur.execute("""SELECT _id, income_label FROM adult""".format(_id)) 
    this_id, income_label = cur.fetchone()
    assert this_id == _id

    greater_than_50k = (income_label == ' >50K')

    cur.execute("""
        BEGIN;
        UPDATE adult 
        SET target = {} 
        WHERE _id = {};
        COMMIT;
    """.format(greater_than_50k, _id)) 

    con.close()

In [71]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id FROM adult WHERE target IS NULL;""") 
this_id, = cur.fetchone()
encode_target(this_id)
con.close()

In [8]:
con, cur = connect_to_postgres()
cur.execute("""
SELECT _id, income_label, target 
FROM adult WHERE _id = {};
""".format(this_id)) 
print(this_id, cur.fetchone())
con.close()

11 (11, ' >50K', True)


In [9]:
con, cur = connect_to_postgres()
cur.execute("""SELECT DISTINCT(workclass) from adult;""") 
workclass = ["workclass"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(education) from adult;""") 
education = ["education"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(marital_status) from adult;""") 
marital_status = ["marital_status"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(occupation) from adult;""") 
occupation = ["occupation"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(relationship) from adult;""") 
relationship = ["relationship"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(race) from adult;""") 
race = ["race"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(gender) from adult;""") 
gender = ["gender"+token[0] for token in cur.fetchall()]
cur.execute("""SELECT DISTINCT(native_country) from adult;""") 
native_country = ["native_country"+token[0] for token in cur.fetchall()]
con.close()

In [9]:
one_hot_labels = workclass + \
                    education + \
                    marital_status + \
                    occupation + \
                    relationship + \
                    race + \
                    gender + \
                    native_country

In [10]:
n_categorical = len(one_hot_labels)

In [12]:
n_categorical

102

In [14]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _ID, * FROM adult;""")
this_row = list(cur.fetchone())
this_id = this_row[0]
this_row_numeric = [this_row[1], this_row[3], this_row[5], this_row[11], this_row[12], this_row[13]]
this_row_categorical = [
    "workclass" + this_row[2],
    "education" + this_row[4],
    "marital_status" + this_row[6],
    "occupation" + this_row[7],
    "relationship" + this_row[8],
    "race" + this_row[9],
    "gender" + this_row[10],
    "native_country" + this_row[14]
]

In [16]:
this_id, this_row_numeric, this_row_categorical

(1,
 [39, 77516, 13, 2174, 0, 40],
 ['workclass State-gov',
  'education Bachelors',
  'marital_status Never-married',
  'occupation Adm-clerical',
  'relationship Not-in-family',
  'race White',
  'gender Male',
  'native_country United-States'])

In [19]:
encoded_categorical = [0]*n_categorical

In [20]:
one_hot_indices = [one_hot_labels.index(attr) for attr in this_row_categorical]

In [21]:
for i in one_hot_indices:
    encoded_categorical[i] = 1   

In [22]:
this_row_vector = np.array(this_row_numeric + encoded_categorical)
this_row_vector

array([   39, 77516,    13,  2174,     0,    40,     0,     1,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     1,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     1,     0,     0,     0,     1,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1,     0,     1,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     1,
           0,     0,     0,     0,     0,     0,     0,     0,     0])

In [15]:
this_row

[1,
 39,
 ' State-gov',
 77516,
 ' Bachelors',
 13,
 ' Never-married',
 ' Adm-clerical',
 ' Not-in-family',
 ' White',
 ' Male',
 2174,
 0,
 40,
 ' United-States',
 ' <=50K',
 None,
 1,
 None]

In [23]:
import psycopg2 as pg2

In [24]:
pg_vector = pg2.Binary(this_row_vector)

In [28]:
con, cur = connect_to_postgres()
cur.execute("""
BEGIN; 
UPDATE adult SET vector = {} 
WHERE _id = {};
COMMIT;""".format(pg_vector, this_id))
con.close()

In [31]:
con, cur = connect_to_postgres()
cur.execute("""
SELECT vector FROM adult WHERE _id = {};
""".format(this_id))
res = cur.fetchall()[0]

con.close()

In [35]:
np.frombuffer(res[0], dtype=int)

array([   39, 77516,    13,  2174,     0,    40,     0,     1,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     1,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     1,     0,     0,     0,     1,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1,     0,     1,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     1,
           0,     0,     0,     0,     0,     0,     0,     0,     0])